In [0]:
# IMPORTING DATA ANALYSIS AND VISUALIZATION LIBRARIES
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt

# SPARK LIBRARIES 
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinMaxScaler, VectorAssembler, StandardScaler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# MODELING 
# from prophet import Prophet
import logging
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

# WARNINGS AND LOGISTICS 
#   Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")
#   disable informational messages from prophet
logging.getLogger('py4j').setLevel(logging.ERROR)

In [0]:
%run ./Snowflake_Connection

In [0]:
sfdt = SnowflakeDataTool()

In [0]:
gl_demand = spark.table('ltf_db.LTF_GOLDEN_LAYER')

In [0]:
# TWO FILES THAT WILL PROVIDE A LEVEL OF GROUP 
pa_zip = spark.table('ltf_db.pa_to_zip_code')
sel_zip = spark.table('ltf_db.sel_to_zip_code')

# display(sel_zip)

In [0]:
grouping_features = sfdt.sql("select DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE as ACTUAL_DATE, \
                        DELIVERY_EXECUTION.SHIPTO_CUSTOMER_ID as CUSTOMER_ID, \
                        DELIVERY_EXECUTION.MATERIAL_ID as MATERIAL_ID, \
                        MATERIAL.BRAND_DESC as BRAND_DESC, \
                        MATERIAL.SWIRE_PACKAGE_CATEGORY_DESCRIPTION, \
                        Customer.SUPER_CHANNEL_DEFINITION, \
                        Customer.ADDRESS_ZIP_CODE as ZIP_CODE \
                    from \
                        db_bi_p_edw.general_use_bas.bas_fact_delivery_execution as DELIVERY_EXECUTION \
                    inner join \
                        DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_CUSTOMER_MASTER as CUSTOMER on (DELIVERY_EXECUTION.SHIPTO_CUSTOMER_ID = CUSTOMER.customer_number) \
                    inner join \
                        DB_SWIRE_BI_P_EDW.TRANSFORMED.DIM_MDM_MATERIAL_MASTER as MATERIAL on (DELIVERY_EXECUTION.MATERIAL_SK = MATERIAL.MATERIAL_SK) \
                    where \
                        DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE >= date('2019-01-01') \
                    and \
                        DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE < CURRENT_DATE() \
                    and \
                        DELIVERY_EXECUTION.ACTUAL_GOODS_MOVEMENT_DATE != date('9999-12-31') \
                    and \
                        MATERIAL.MATERIAL_TYPE in ('ZFER') \
                    and \
                        CUSTOMER.DIVISION = 'SA'")
grouping_features = grouping_features.orderBy('ACTUAL_DATE')
# display(grouping_features)

In [0]:
sel_zip_grouping_feat_channel = grouping_features.join(sel_zip, \
                                                   on=(grouping_features.ZIP_CODE == sel_zip.Zip), \
                                                   how= 'inner') \
                                            .select(grouping_features.ACTUAL_DATE, grouping_features.CUSTOMER_ID, grouping_features.MATERIAL_ID, (sel_zip.SEL_Cluster).alias('SEL_CLUSTER'), grouping_features.BRAND_DESC, grouping_features.SWIRE_PACKAGE_CATEGORY_DESCRIPTION, grouping_features.SUPER_CHANNEL_DEFINITION)

pa_zip_grouping_feat_channel = grouping_features.join(pa_zip, \
                                                   on=(grouping_features.ZIP_CODE == pa_zip.ADDRESS_ZIP_CODE), \
                                                   how= 'inner') \
                                            .select(grouping_features.ACTUAL_DATE, grouping_features.CUSTOMER_ID, grouping_features.MATERIAL_ID, (pa_zip.MARKET).alias('PA_MARKET'), grouping_features.BRAND_DESC, grouping_features.SWIRE_PACKAGE_CATEGORY_DESCRIPTION, grouping_features.SUPER_CHANNEL_DEFINITION)

In [0]:
display(pa_zip_grouping_feat_channel)

ACTUAL_DATE,CUSTOMER_ID,MATERIAL_ID,PA_ZIP,BRAND_DESC,SWIRE_PACKAGE_CATEGORY_DESCRIPTION,SUPER_CHANNEL_DEFINITION
2020-06-29,0600241534,000000000000119826,85366,COCA-COLA,SSD NR 20Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000121765,85366,COCA-COLA ZERO,SSD NR 20Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000119826,85366,COCA-COLA,SSD NR 20Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000117803,85366,DR PEPPER-CS,SSD NR 20Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000103029,85366,SPRITE,SSD NR 20Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000133129,85366,MONSTER-KO,ENG MNSTR 15.5/16Z 24CT,FULL SERVICE
2020-11-12,0600241534,000000000000112259,85366,DASANI,WTR DASANI 20Z 24CT,FULL SERVICE
2020-11-16,0600241534,000000000000117803,85366,DR PEPPER-CS,SSD NR 20Z 24CT,FULL SERVICE
2020-11-16,0600241534,000000000000119826,85366,COCA-COLA,SSD NR 20Z 24CT,FULL SERVICE
2020-11-16,0600241534,000000000000103029,85366,SPRITE,SSD NR 20Z 24CT,FULL SERVICE


In [0]:
# sel_zip_grouping_feat_channel.write.format("delta").mode('overwrite').saveAsTable('ltf_db.sel_zip_grouping_feat_channel')
# pa_zip_grouping_feat_channel.write.mode('overwrite').saveAsTable('ltf_db.pa_zip_grouping_feat_channel')
 

In [0]:
gl_pa_brand = gl_demand.join(pa_zip_grouping_feat_channel, \
                            on=((gl_demand.CUSTOMER_ID == pa_zip_grouping_feat_channel.CUSTOMER_ID) & (gl_demand.MATERIAL_ID == pa_zip_grouping_feat_channel.MATERIAL_ID)), \
                             how='inner') \
                                 .select(gl_demand.TS_DATE, gl_demand.CUSTOMER_ID, gl_demand.MATERIAL_ID, pa_zip_grouping_feat_channel.SUPER_CHANNEL_DEFINITION, pa_zip_grouping_feat_channel.PA_MARKET, pa_zip_grouping_feat_channel.BRAND_DESC, gl_demand.ORDERED_WEEKLY_SUM).orderBy('TS_DATE')

display(gl_pa_brand)

TS_DATE,CUSTOMER_ID,MATERIAL_ID,SUPER_CHANNEL_DEFINITION,PA_MARKET,BRAND_DESC,ORDERED_WEEKLY_SUM
2019-01-04,0500292145,000000000000132296,ON PREMISE,179,GLACEAU SMARTWATER,180.000
2019-01-04,0500300499,000000000000142720,SUPER MARKET,794,GLACEAU VWTR ZERO-KO,8.000
2019-01-04,0500292145,000000000000132838,ON PREMISE,179,GOLD PEAK,10.000
2019-01-04,0500313540,000000000000115314,CONVENIENCE RETAIL,866,FANTA,1.000
2019-01-04,0500300499,000000000000146125,SUPER MARKET,794,SEAGRAMS-KO,16.000
2019-01-04,0500300499,000000000000142720,SUPER MARKET,794,GLACEAU VWTR ZERO-KO,8.000
2019-01-04,0500313540,000000000000132607,CONVENIENCE RETAIL,866,DIET DR PEPPER-CS,1.000
2019-01-04,0500276754,000000000000112309,VALUE,212,DASANI,1.000
2019-01-04,0500300499,000000000000133257,SUPER MARKET,794,DIET DR PEPPER-CS,16.000
2019-01-04,0500300499,000000000000142720,SUPER MARKET,794,GLACEAU VWTR ZERO-KO,8.000


In [0]:
from pyspark.sql.functions import countDistinct

# Group by the specified columns and count the distinct groups
num_groups = gl_pa_brand.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "BRAND_DESC") \
               .agg(countDistinct("CUSTOMER_ID", "MATERIAL_ID")) \
               .count()


# Print the number of distinct groups
print(f"There are {num_groups} unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, PA_MARKET AND BRAND.")

There are 7091 unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, PA_MARKET AND BRAND.


In [0]:
from pyspark.sql.functions import count, countDistinct

grouped_df = gl_pa_brand.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "BRAND_DESC")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

for row in unique_dates_df.collect():
    group = (row.SUPER_CHANNEL_DEFINITION, row.PA_MARKET, row.BRAND_DESC)
    num_unique_dates = row.num_unique_dates
    num_rows = row.num_rows
    print(f"Group {group} has {num_rows} rows and {num_unique_dates} unique dates.")

Group ('CONVENIENCE RETAIL', 316, 'GLACEAU SMARTWATER') has 244252 rows and 215 unique dates.
Group ('ON PREMISE', 793, 'BODYARMOR EDGE-KO') has 40 rows and 3 unique dates.
Group ('SUPER MARKET', 220, 'TOPO CHICO-KO') has 118 rows and 8 unique dates.
Group ('CONVENIENCE RETAIL', 866, 'COCA-COLA ENERGY') has 11294 rows and 78 unique dates.
Group ('DRUG', 259, "DUNKIN'") has 1860 rows and 34 unique dates.
Group ('ON PREMISE', 622, 'COCA-COLA LIGHT/DIET COKE') has 75767 rows and 209 unique dates.
Group ('DRUG', 310, 'SPRITE') has 6091 rows and 75 unique dates.
Group ('DRUG', 891, 'SPRITE') has 22785 rows and 164 unique dates.
Group ('ON PREMISE', 674, 'SPRITE ZERO/DIET/LIGHT') has 267 rows and 18 unique dates.
Group ('ON PREMISE', 531, 'GLACEAU SMARTWATER') has 17495 rows and 120 unique dates.
Group ('VALUE', 585, 'REIGN-KO') has 60842 rows and 178 unique dates.
Group ('CONVENIENCE RETAIL', 478, "DUNKIN'") has 17198 rows and 109 unique dates.
Group ('CONVENIENCE RETAIL', 585, 'COCA-COLA')

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates_pa_brand = gl_pa_brand.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df_pa_brand = gl_pa_brand.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "BRAND_DESC")
unique_dates_df_pa_brand = grouped_df_pa_brand.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Calculate the percentage of unique dates to total dates for each group
percentage_df_pa_brand = unique_dates_df_pa_brand.withColumn("percentage", unique_dates_df_pa_brand.num_unique_dates / lit(total_unique_dates_pa_brand) * 100)

# Display the summary information
display(percentage_df_pa_brand)

SUPER_CHANNEL_DEFINITION,PA_MARKET,BRAND_DESC,num_unique_dates,num_rows,percentage
ON PREMISE,531,GLACEAU SMARTWATER,120,17495,55.55555555555556
ON PREMISE,622,COCA-COLA LIGHT/DIET COKE,209,75767,96.75925925925925
HYPER,277,REIGN-KO,201,129625,93.05555555555556
VALUE,585,REIGN-KO,178,60842,82.4074074074074
CONVENIENCE RETAIL,316,GLACEAU SMARTWATER,215,244252,99.53703703703704
ON PREMISE,674,AHA-KO,37,1592,17.12962962962963
DRUG,891,SPRITE,164,22785,75.92592592592592
DRUG,310,SPRITE,75,6091,34.72222222222222
CONVENIENCE RETAIL,478,DUNKIN',109,17198,50.46296296296296
ALL OTHER,212,MCCAFE,3,9,1.3888888888888888


Output can only be rendered in Databricks

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates = gl_pa_brand.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df = gl_pa_brand.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "BRAND_DESC")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Count the number of groups that contain the full total amount of unique dates
full_date_count = unique_dates_df.filter(unique_dates_df.num_unique_dates == total_unique_dates).count()

# Display the summary information
print(f"There are {full_date_count} groups that contain the full total amount of unique dates.")

There are 512 groups that contain the full total amount of unique dates.


In [0]:
from pyspark.sql.functions import countDistinct, avg

# Group the DataFrame by the specified columns and count the number of unique dates in each group
grouped_df = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"))

# Calculate the average number of unique dates per grouping
avg_unique_dates = unique_dates_df.select(avg("num_unique_dates")).collect()[0][0]

# Display the summary information
print(f"The average number of unique dates per grouping is {avg_unique_dates:.2f}.")

The average number of unique dates per grouping is 78.41.


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
gl_pa_package = gl_demand.join(pa_zip_grouping_feat_channel, \
                            on=((gl_demand.CUSTOMER_ID == pa_zip_grouping_feat_channel.CUSTOMER_ID) & (gl_demand.MATERIAL_ID == pa_zip_grouping_feat_channel.MATERIAL_ID)), \
                             how='inner') \
                                 .select(gl_demand.TS_DATE, gl_demand.CUSTOMER_ID, gl_demand.MATERIAL_ID, pa_zip_grouping_feat_channel.SUPER_CHANNEL_DEFINITION, pa_zip_grouping_feat_channel.PA_MARKET, pa_zip_grouping_feat_channel.SWIRE_PACKAGE_CATEGORY_DESCRIPTION, gl_demand.ORDERED_WEEKLY_SUM)

display(gl_pa_package)

TS_DATE,CUSTOMER_ID,MATERIAL_ID,SUPER_CHANNEL_DEFINITION,PA_MARKET,SWIRE_PACKAGE_CATEGORY_DESCRIPTION,ORDERED_WEEKLY_SUM
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-09-24,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,5.000
2021-10-08,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,4.000
2021-10-08,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,4.000
2021-10-08,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,4.000
2021-10-08,0500266792,000000000000119826,ON PREMISE,212,SSD NR 20Z 24CT,4.000


In [0]:
from pyspark.sql.functions import countDistinct

# Group by the specified columns and count the distinct groups
num_groups = gl_pa_package.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION") \
               .agg(countDistinct("CUSTOMER_ID", "MATERIAL_ID")) \
               .count()

# Print the number of distinct groups
print(f"There are {num_groups} unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, PA_MARKET AND PACKAGE.")

There are 11784 unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, PA_MARKET AND PACKAGE.


In [0]:
from pyspark.sql.functions import count, countDistinct

grouped_df = gl_pa_package.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

for row in unique_dates_df.collect():
    group = (row.SUPER_CHANNEL_DEFINITION, row.PA_MARKET, row.SWIRE_PACKAGE_CATEGORY_DESCRIPTION)
    num_unique_dates = row.num_unique_dates
    num_rows = row.num_rows
    print(f"Group {group} has {num_rows} rows and {num_unique_dates} unique dates.")

Group ('ON PREMISE', 277, 'ENG MNSTR JAVA 15Z 12CT') has 178 rows and 25 unique dates.
Group ('CONVENIENCE RETAIL', 534, 'TEA PEACE TEA 23Z 12CT') has 301226 rows and 209 unique dates.
Group ('SUPER MARKET', 294, 'SSD CANS 7.5Z 10PK 3CT') has 103974 rows and 197 unique dates.
Group ('ON PREMISE', 277, 'TEA PEACE TEA 23Z 12CT') has 24190 rows and 140 unique dates.
Group ('CONVENIENCE RETAIL', 259, 'SSD CANS FLAV 12Z 12PK 2CT') has 68849 rows and 184 unique dates.
Group ('CONVENIENCE RETAIL', 893, 'JCE TY 10Z 12CT') has 68694 rows and 181 unique dates.
Group ('ON PREMISE', 729, 'MLK COREPWR 11.5/14Z 12CT') has 85 rows and 9 unique dates.
Group ('ON PREMISE', 794, 'OTH CO2 FULL') has 6866 rows and 116 unique dates.
Group ('SUPER MARKET', 729, 'SSD NR GLASS 355ML 24PK') has 136 rows and 5 unique dates.
Group ('CONVENIENCE RETAIL', 220, 'PWD PA 20Z 24CT') has 14943 rows and 112 unique dates.
Group ('MASS', 259, 'SSD NR FLAV 2L 8CT') has 1920 rows and 39 unique dates.
Group ('HYPER', 729, 'E

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates_pa_package = gl_pa_package.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df_pa_package = gl_pa_package.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df_pa_package = grouped_df_pa_package.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Calculate the percentage of unique dates to total dates for each group
percentage_df_pa_package = unique_dates_df_pa_package.withColumn("percentage", unique_dates_df_pa_package.num_unique_dates / lit(total_unique_dates_pa_package) * 100)

# Display the summary information
display(percentage_df_pa_package)

SUPER_CHANNEL_DEFINITION,PA_MARKET,SWIRE_PACKAGE_CATEGORY_DESCRIPTION,num_unique_dates,num_rows,percentage
CONVENIENCE RETAIL,277,WTR SW 700ML PLUS 12CT,93,17036,43.05555555555556
VALUE,220,SSD NR 1.25L 12CT,171,34389,79.16666666666666
CONVENIENCE RETAIL,893,JCE TY 10Z 12CT,181,68694,83.79629629629629
CONVENIENCE RETAIL,866,JCE HUBERT 16Z 12CT,106,48730,49.074074074074076
CONVENIENCE RETAIL,534,TEA PEACE TEA 23Z 12CT,209,301226,96.75925925925925
CONVENIENCE RETAIL,794,WTR BA 1L 12CT,43,6322,19.90740740740741
ON PREMISE,277,TEA PEACE TEA 23Z 12CT,140,24190,64.81481481481481
SUPER MARKET,294,ENG REIGN 16Z 12CT,124,15165,57.407407407407405
SUPER MARKET,294,SSD CANS 7.5Z 10PK 3CT,197,103974,91.20370370370371
DRUG,259,TEA GOLD PEAK 18.5Z 12CT,38,3322,17.59259259259259


Output can only be rendered in Databricks

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates = gl_pa_package.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df = gl_pa_package.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Count the number of groups that contain the full total amount of unique dates
full_date_count = unique_dates_df.filter(unique_dates_df.num_unique_dates == total_unique_dates).count()

# Display the summary information
print(f"There are {full_date_count} groups that contain the full total amount of unique dates.")

There are 304 groups that contain the full total amount of unique dates.


In [0]:
from pyspark.sql.functions import countDistinct, avg

# Group the DataFrame by the specified columns and count the number of unique dates in each group
grouped_df = gl_pa_package.groupBy("SUPER_CHANNEL_DEFINITION", "PA_MARKET", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"))

# Calculate the average number of unique dates per grouping
avg_unique_dates = unique_dates_df.select(avg("num_unique_dates")).collect()[0][0]

# Display the summary information
print(f"The average number of unique dates per grouping is {avg_unique_dates:.2f}.")

The average number of unique dates per grouping is 63.92.


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
gl_sel_brand = gl_demand.join(sel_zip_grouping_feat_channel, \
                            on=((gl_demand.CUSTOMER_ID == sel_zip_grouping_feat_channel.CUSTOMER_ID) & (gl_demand.MATERIAL_ID == sel_zip_grouping_feat_channel.MATERIAL_ID)), \
                             how='inner') \
                                 .select(gl_demand.TS_DATE, gl_demand.CUSTOMER_ID, gl_demand.MATERIAL_ID, sel_zip_grouping_feat_channel.SUPER_CHANNEL_DEFINITION, sel_zip_grouping_feat_channel.SEL_CLUSTER, sel_zip_grouping_feat_channel.BRAND_DESC, gl_demand.ORDERED_WEEKLY_SUM)

display(gl_sel_brand)

TS_DATE,CUSTOMER_ID,MATERIAL_ID,SUPER_CHANNEL_DEFINITION,SEL_CLUSTER,BRAND_DESC,ORDERED_WEEKLY_SUM
2021-01-29,0500246183,000000000000410069,ON PREMISE,SEL-4,COCA-COLA FUNCTIONAL,4.000
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SPRITE,0.083


In [0]:
from pyspark.sql.functions import countDistinct

# Group by the specified columns and count the distinct groups
num_groups = gl_sel_brand.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "BRAND_DESC") \
               .agg(countDistinct("CUSTOMER_ID", "MATERIAL_ID")) \
               .count()

# Print the number of distinct groups
print(f"There are {num_groups} unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, SEL_CLUSTER AND BRAND.")

There are 1573 unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, SEL_CLUSTER AND BRAND.


In [0]:
from pyspark.sql.functions import count, countDistinct

grouped_df = gl_sel_brand.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "BRAND_DESC")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

for row in unique_dates_df.collect():
    group = (row.SUPER_CHANNEL_DEFINITION, row.SEL_CLUSTER, row.BRAND_DESC)
    num_unique_dates = row.num_unique_dates
    num_rows = row.num_rows
    print(f"Group {group} has {num_rows} rows and {num_unique_dates} unique dates.")

Group ('SUPER MARKET', 'SEL-2', 'DIET DR PEPPER-CS') has 1238366 rows and 216 unique dates.
Group ('FULL SERVICE', 'SEL-2', 'DIET DR PEPPER-CS') has 132752 rows and 216 unique dates.
Group ('ON PREMISE', 'SEL-2', 'GLACEAU VWTR ZERO-KO') has 107352 rows and 213 unique dates.
Group ('HYPER', 'SEL-4', 'FAIRLIFE MILK-KO') has 7272 rows and 42 unique dates.
Group ('CONVENIENCE RETAIL', 'SEL-3', 'TOPO CHICO-KO') has 3818 rows and 95 unique dates.
Group ('HYPER', 'SEL-1', 'DIET DR PEPPER-CS') has 4693631 rows and 216 unique dates.
Group ('DRUG', 'SEL-2', 'FANTA') has 113502 rows and 216 unique dates.
Group ('SUPER MARKET', 'SEL-2', 'GLACEAU VITAMINWATER-KO') has 824924 rows and 216 unique dates.
Group ('CONVENIENCE RETAIL', 'SEL-3', 'BODYARMOR SPORTWATER-KO') has 33472 rows and 46 unique dates.
Group ('ON PREMISE', 'SEL-4', 'DASANI') has 565788 rows and 216 unique dates.
Group ('DRUG', 'SEL-2', 'GLACEAU VWTR ZERO-KO') has 46937 rows and 202 unique dates.
Group ('ON PREMISE', 'SEL-1', 'COCA-CO

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates_sel_brand = gl_sel_brand.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df_sel_brand = gl_sel_brand.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "BRAND_DESC")
unique_dates_df_sel_brand = grouped_df_sel_brand.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Calculate the percentage of unique dates to total dates for each group
percentage_df_sel_brand = unique_dates_df_sel_brand.withColumn("percentage", unique_dates_df_sel_brand.num_unique_dates / lit(total_unique_dates_sel_brand) * 100)

# Display the summary information
display(percentage_df_sel_brand)


SUPER_CHANNEL_DEFINITION,SEL_CLUSTER,BRAND_DESC,num_unique_dates,num_rows,percentage
ON PREMISE,SEL-2,GLACEAU VWTR ZERO-KO,213,107352,98.61111111111111
SUPER MARKET,SEL-2,DIET DR PEPPER-CS,216,1238366,100.0
FULL SERVICE,SEL-2,DIET DR PEPPER-CS,216,132752,100.0
CONVENIENCE RETAIL,SEL-3,TOPO CHICO-KO,95,3818,43.98148148148148
HYPER,SEL-4,FAIRLIFE MILK-KO,42,7272,19.444444444444446
HYPER,SEL-1,DIET DR PEPPER-CS,216,4693631,100.0
DRUG,SEL-2,FANTA,216,113502,100.0
CONVENIENCE RETAIL,SEL-3,BODYARMOR SPORTWATER-KO,46,33472,21.296296296296298
ON PREMISE,SEL-4,DASANI,216,565788,100.0
DRUG,SEL-2,GLACEAU VWTR ZERO-KO,202,46937,93.51851851851852


Output can only be rendered in Databricks

Output can only be rendered in Databricks

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates = gl_sel_brand.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df = gl_sel_brand.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "BRAND_DESC")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Count the number of groups that contain the full total amount of unique dates
full_date_count = unique_dates_df.filter(unique_dates_df.num_unique_dates == total_unique_dates).count()

# Display the summary information
print(f"There are {full_date_count} groups that contain the full total amount of unique dates.")

There are 356 groups that contain the full total amount of unique dates.


In [0]:
from pyspark.sql.functions import countDistinct, avg

# Group the DataFrame by the specified columns and count the number of unique dates in each group
grouped_df = gl_sel_brand.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "BRAND_DESC")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"))

# Calculate the average number of unique dates per grouping
avg_unique_dates = unique_dates_df.select(avg("num_unique_dates")).collect()[0][0]

# Display the summary information
print(f"The average number of unique dates per grouping is {avg_unique_dates:.2f}.")

The average number of unique dates per grouping is 118.95.


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
gl_sel_package = gl_demand.join(sel_zip_grouping_feat_channel, \
                            on=((gl_demand.CUSTOMER_ID == sel_zip_grouping_feat_channel.CUSTOMER_ID) & (gl_demand.MATERIAL_ID == sel_zip_grouping_feat_channel.MATERIAL_ID)), \
                             how='inner') \
                                 .select(gl_demand.TS_DATE, gl_demand.CUSTOMER_ID, gl_demand.MATERIAL_ID, sel_zip_grouping_feat_channel.SUPER_CHANNEL_DEFINITION, sel_zip_grouping_feat_channel.SEL_CLUSTER, pa_zip_grouping_feat_channel.SWIRE_PACKAGE_CATEGORY_DESCRIPTION, gl_demand.ORDERED_WEEKLY_SUM)

display(gl_sel_package)

TS_DATE,CUSTOMER_ID,MATERIAL_ID,SUPER_CHANNEL_DEFINITION,SEL_CLUSTER,SWIRE_PACKAGE_CATEGORY_DESCRIPTION,ORDERED_WEEKLY_SUM
2021-01-29,0500246183,000000000000410069,ON PREMISE,SEL-4,CFE COKE 12Z 12CT,4.000
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083
2022-02-18,0500264326,000000000000103029,FULL SERVICE,SEL-4,SSD NR 20Z 24CT,0.083


In [0]:
from pyspark.sql.functions import countDistinct

# Group by the specified columns and count the distinct groups
num_groups = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION") \
               .agg(countDistinct("CUSTOMER_ID", "MATERIAL_ID")) \
               .count()

# Print the number of distinct groups
print(f"There are {num_groups} unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, SEL_CLUSTER AND PACKAGE.")


There are 3331 unique grouping combinations using SUPER_CHANNEL_DESCRIPTION, SEL_CLUSTER AND PACKAGE.


In [0]:
from pyspark.sql.functions import count, countDistinct

grouped_df = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

for row in unique_dates_df.collect():
    group = (row.SUPER_CHANNEL_DEFINITION, row.SEL_CLUSTER, row.SWIRE_PACKAGE_CATEGORY_DESCRIPTION)
    num_unique_dates = row.num_unique_dates
    num_rows = row.num_rows
    print(f"Group {group} has {num_rows} rows and {num_unique_dates} unique dates.")


Group ('HYPER', 'SEL-1', 'SSD NR GLASS 8Z 4CT') has 300481 rows and 207 unique dates.
Group ('HYPER', 'SEL-2', 'PWD PA 32Z 15CT') has 160932 rows and 56 unique dates.
Group ('ON PREMISE', 'SEL-2', 'ENG COKE 12Z 24CT') has 5509 rows and 76 unique dates.
Group ('SUPER MARKET', 'SEL-2', 'WTR DASANI 12Z 8PK 3CT') has 76687 rows and 177 unique dates.
Group ('SUPER MARKET', 'SEL-3', 'SSD NR GLASS 8Z 4CT') has 44190 rows and 185 unique dates.
Group ('DRUG', 'SEL-1', 'SSD NR .5L 6PK 4CT') has 116037 rows and 205 unique dates.
Group ('DRUG', 'SEL-4', 'EWTR VW 20Z 12CT') has 42540 rows and 196 unique dates.
Group ('MASS', 'SEL-1', 'WTR DASANI 20Z 24CT') has 17110 rows and 118 unique dates.
Group ('ON PREMISE', 'SEL-4', 'WTR ZICO 16.9Z 12CT') has 2299 rows and 71 unique dates.
Group ('SUPER MARKET', 'SEL-2', 'PWD PA 20Z 8PK 3CT') has 576310 rows and 208 unique dates.
Group ('FULL SERVICE', 'SEL-1', 'WTR DASANI .3L 24CT') has 93154 rows and 165 unique dates.
Group ('VALUE', 'SEL-4', 'SPT BA 28Z 12

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates_sel_package = gl_sel_package.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df_sel_package = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df_sel_package = grouped_df_sel_package.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Calculate the percentage of unique dates to total dates for each group
percentage_df_sel_package = unique_dates_df_sel_package.withColumn("percentage", unique_dates_df_sel_package.num_unique_dates / lit(total_unique_dates_sel_package) * 100)

# Display the summary information
display(percentage_df_sel_package)


SUPER_CHANNEL_DEFINITION,SEL_CLUSTER,SWIRE_PACKAGE_CATEGORY_DESCRIPTION,num_unique_dates,num_rows,percentage
ON PREMISE,SEL-4,JCE MM 12Z/450ML 24CT,207,79975,99.04306220095694
HYPER,SEL-1,CFE COKE 12Z 12CT,99,29239,47.368421052631575
DRUG,SEL-1,SSD NR .5L 6PK 4CT,205,116037,98.08612440191388
SUPER MARKET,SEL-3,SSD NR GLASS 8Z 4CT,185,44190,88.51674641148325
DRUG,SEL-4,EWTR VW 20Z 12CT,196,42540,93.77990430622009
HYPER,SEL-1,SSD NR GLASS 8Z 4CT,207,300481,99.04306220095694
SUPER MARKET,SEL-2,WTR DASANI 12Z 8PK 3CT,177,76687,84.688995215311
ON PREMISE,SEL-2,ENG COKE 12Z 24CT,76,5509,36.36363636363637
MASS,SEL-1,WTR DASANI 20Z 24CT,118,17110,56.45933014354066
VALUE,SEL-1,PWD PA 28Z 15CT,154,118237,73.68421052631578


Output can only be rendered in Databricks

In [0]:
from pyspark.sql.functions import count, countDistinct, lit

# Count the total number of unique dates in the TS_DATE column
total_unique_dates = gl_sel_package.select(countDistinct("TS_DATE")).collect()[0][0]

# Group the DataFrame by the specified columns and count the number of unique dates and rows
grouped_df = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"), count("*").alias("num_rows"))

# Count the number of groups that contain the full total amount of unique dates
full_date_count = unique_dates_df.filter(unique_dates_df.num_unique_dates == total_unique_dates).count()

# Display the summary information
print(f"There are {full_date_count} groups that contain the full total amount of unique dates.")


There are 290 groups that contain the full total amount of unique dates.


In [0]:
from pyspark.sql.functions import countDistinct, avg

# Group the DataFrame by the specified columns and count the number of unique dates in each group
grouped_df = gl_sel_package.groupBy("SUPER_CHANNEL_DEFINITION", "SEL_CLUSTER", "SWIRE_PACKAGE_CATEGORY_DESCRIPTION")
unique_dates_df = grouped_df.agg(countDistinct("TS_DATE").alias("num_unique_dates"))

# Calculate the average number of unique dates per grouping
avg_unique_dates = unique_dates_df.select(avg("num_unique_dates")).collect()[0][0]

# Display the summary information
print(f"The average number of unique dates per grouping is {avg_unique_dates:.2f}.")

The average number of unique dates per grouping is 78.41.


CUSTOMER dimensions: SEL (ZIP) // Precision Area (ZIP)// Super Channel 

MATERIAL dimensions: BRAND // Package // 

gl_pa_brand == SUPER CHANNEL + PA ZIP + BRAND //

gl_pa_package == SUPER CHANNEL + PA ZIP + PACKAGE //

gl_sel_brand == SUPER CHANNEL + SEL ZIP + BRAND //

gl_sel_package == SUPER CHANNEL + SEL ZIP + PACKAGE //